Data obtained from:
https://www.sciencedirect.com/science/article/abs/pii/S0957417420302323?casa_token=KAfym3lH_REAAAAA:VvMAXW-RJ7-ZNRJdoEhmO5zAdSXdmx4t6hCY1kifJYm_Q98DOlWMDqv6GL8ZFNDdrKfB-TRb08Q#sec0002